In [217]:
import pandas as pd
import geopandas as gpd
import os
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [218]:
path_adm = r'Z:\Masterfile\1. Shapefile\1. Population and Administrative Boundaries\3. New Area'
path_masterfile = r'C:\Users\User\OneDrive - Migo (1)\Documents\Migo\Data Project\Masterfile'
path_performance = r'Z:\Masterfile\3. Files\2. Performance Q4'

In [219]:
# Reading Files
region = gpd.read_file(os.path.join(path_adm,'new_area.gpkg'))
masterfile = pd.read_excel(os.path.join(path_masterfile, 'Store Score Masterfile.xlsx'), sheet_name='Masterfile New')
performance = pd.read_excel(os.path.join(path_performance, 'Q4_Performance_alltochurned.xlsx'), sheet_name='All_Store')

In [220]:
performance.columns

Index(['Store ID', 'Store Name', 'Service Center', 'City', 'Kecamatan',
       'Siting Type', 'Deployment Date', 'SME', 'Total Gross Revenue',
       'Total Unique Watchers', 'Count Normal Month', 'Count Watchlist Month',
       'Count Focus Month'],
      dtype='object')

In [221]:
#Checking columns
region.columns

Index(['OBJECTID', 'PROVINCE', 'CITY', 'KECA', 'PCK', 'SUM_AREA', 'SUM_POP',
       'SUM_DENSIT', 'Shape_Leng', 'Shape_Area', 'Area', 'geometry'],
      dtype='object')

In [222]:
# Join masterfile with region
region['new_id'] = region['CITY'].str.lower().str.strip() +" "+region['KECA'].str.lower().str.strip().str.replace(" ","")
masterfile['new_id'] = masterfile['City'].str.lower().str.strip() + " "+ masterfile['Kecamatan'].str.lower().str.strip().str.replace(" ","")
masterfile_join = masterfile.merge(region, how ='left', on = 'new_id')
masterfile_join.columns

Index(['Area_Wilayah', 'Nama_Toko', 'Jenis_Toko', 'Alamat', 'Lintang', 'Bujur',
       'Kecamatan', 'City', 'Province', 'Store_ID', 'store_join2',
       'sitting_type', 'status3', 'Store Score', 'Pitch Team',
       'Operational Hours', 'Store Staff', 'Store Tenure Type', 'gov', 'cvs',
       'market', 'School', 'religion', 'transport', 'health_center',
       'future_plan_to_stay_in_month', 'store_directional_visibility',
       'distance_between_store_and_street_in_front_of_the_store_meter',
       'store_wide_frontage_meter', 'average_store_visitors_min',
       'average_store_visitors_max', 'store_awning_and_sheltering',
       'consumer_behavior', 'Since when (year)', 'Average daily visitors',
       'ZMP', 'Since When Host Stay', 'Store Day Off',
       'Store Remaining Contract', 'Electric', 'Pitch_Date', 'Total_Active_ID',
       'Total_Watch_ID', 'POI CB', 'Customer_Variety', 'Customer_Type',
       'Host_Birthday', 'Host_Age', 'new_id', 'OBJECTID', 'PROVINCE', 'CITY',
      

In [223]:
#Join masterfile join with performance
masterfile_join_2 = masterfile_join.merge(performance, how ='left',left_on = 'Store_ID', right_on = 'Store ID')
masterfile_join_2 = masterfile_join_2.rename(columns={'Total Gross Revenue':'gr_q4', 'Total Unique Watchers':'watchers_q4',
                                                      'Kecamatan_x':'Kecamatan', 'City_x' :'City', 'SUM_POP': 'population',
                                                      'status3':'status'})

In [224]:
#Preparing Masterfile to Update
masterfile_join_2 = masterfile_join_2[['Area_Wilayah','Store_ID', 'Nama_Toko','Lintang', 'Bujur', 'Kecamatan', 'City', 'Province',
                         'status','new_id','sitting_type', 'population','gr_q4', 'watchers_q4','Area']]
masterfile_join_2 = masterfile_join_2.fillna(0)
masterfile_join_2 = masterfile_join_2[masterfile_join_2['Store_ID']!=0]
masterfile_join_2 = masterfile_join_2.fillna(0)

# Updating to Google Sheets

In [225]:
#Reading from
spreadsheetId = "1P3mo9DnaZ9Kv3slU7AWSQCBuOJZgKPUMA5WAMhHXypY"
sheetId = "1821994154"

sh = client.open_by_key(spreadsheetId)
worksheet = sh.get_worksheet(2)
worksheet.clear()

{'spreadsheetId': '1P3mo9DnaZ9Kv3slU7AWSQCBuOJZgKPUMA5WAMhHXypY',
 'clearedRange': 'store_masterfile!A1:N13892'}

In [226]:
worksheet.update([masterfile_join_2.columns.values.tolist()] + masterfile_join_2.values.tolist())

{'spreadsheetId': '1P3mo9DnaZ9Kv3slU7AWSQCBuOJZgKPUMA5WAMhHXypY',
 'updatedRange': 'store_masterfile!A1:O2634',
 'updatedRows': 2634,
 'updatedColumns': 15,
 'updatedCells': 39510}